# NDVI Anomaly
Notebook to generate NDVI anomaly, showing changes in NDVI between two time periods. To show actual change two time period of similar seasonality should be used. 

This notebook compares NDVI between two time periods to detect land change. In the case of deforestation, the NDVI values will reduce from (0.6 to 0.9 ... typical for forests) to lower values (<0.6). This change can be detected and used to investigate deforestation or monitor the extent of the land change.

### Import Required Modules

In [ ]:
# jupyteronly
# modules which are only required when working within a jupyter notebook envionment
%load_ext autoreload
%autoreload 2
%matplotlib inline
# Supress Warning 
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import datacube
from matplotlib.cm import RdYlGn, Greens


In [ ]:
import numpy as np
import xarray as xr 
from pyproj import Proj, transform
from datetime import datetime
import dask
from dask.distributed import Client
import odc.algo
from datacube.utils.cog import write_cog

#import datacube utilities
from datacube_utilities.dc_display_map import display_map
from datacube_utilities.dc_mosaic import create_max_ndvi_mosaic, create_median_mosaic
from datacube_utilities.createAOI import create_lat_lon
from datacube_utilities.createindices import NDVI
from datacube_utilities.dc_utilities import write_geotiff_from_xr

CMAP = "Blues"

client = Client('dask-scheduler.dask.svc.cluster.local:8786')

client.get_versions(check=True)
client

### Initiliase

In [ ]:
dc = datacube.Datacube(app='ndvi anomoly')
dc

### Set up Parameters for cube query

In [4]:
# parameters

#set start and end dates for time period of interest
#parameter display_name="Baseline Start Date" description='Start of baseline time period window' datatype=date"
baseline_time_start = '2010-1-1'
#parameter display_name="Baseline End Date" description='End of baseline time period window' datatype=date"
baseline_time_end = '2010-12-31'

#set start and end dates for time period of interest
#parameter display_name="Analysis Start Date" description='Start of analysis time period window' datatype=date"
analysis_time_start = '2015-1-1'
#parameter display_name="Analysis End Date" description='End of analysis time period window' datatype=date"
analysis_time_end = '2015-12-31'

#set resolution
#parameter display_name="Resolution" description="size of pixes" datatype="int" 
res = (30)

#choose sensor
#parameter display_name="Baseline Sensor" description="Satellite to use for baseline." datatype="string" options=["SENTINEL_2", "LANDSAT_4", "LANDSAT_5", "LANDSAT_7", "LANDSAT_8"],
baseline_platform = "LANDSAT_7"
#parameter display_name="Analysis Sensor" description="Satellite to use for analysis." datatype="string" options=["SENTINEL_2", "LANDSAT_4", "LANDSAT_5", "LANDSAT_7", "LANDSAT_8"],
analysis_platform = "LANDSAT_8"

#parameter which determines what water threshold we accept as being water. Restrict to be between 0 and 1. 
#parameter display_name="Water Threshold" description="The value for how strinct water mask should be, value of 0 equates to always land, value of 1 always water" datatype="float" 
waterThresh = 0.3


#area of interest: load in as wkt
#parameter display_name="Area of Interest" description="The area for which the product is required." datatype="string",
#aoi_wkt = "POLYGON((178.98101806642 -17.592544555664, 179.03903961183 -17.593231201171, 179.03903961183 -17.66258239746, 178.97998809815 -17.661209106445, 178.98101806642 -17.592544555664))"
#aoi_wkt = "POLYGON ((177.62557983398438 -17.590848708679893, 177.77372360229492 -17.590848708679893, 177.77372360229492 -17.488875828028657, 177.62557983398438 -17.488875828028657, 177.62557983398438 -17.590848708679893))"
aoi_wkt = "POLYGON((178.51212272613253 -18.079025694549387,178.57289085357397 -18.079025694549387,178.57289085357397 -18.15113922408976,178.51212272613253 -18.15113922408976,178.51212272613253 -18.079025694549387))"
#Sigatoka
#aoi_wkt = "POLYGON((177.5029362281993 -18.156155117453817,177.53100286333114 -18.156155117453817,177.53100286333114 -18.18062057733904,177.5029362281993 -18.18062057733904,177.5029362281993 -18.156155117453817))"
#aoi_wkt = "POLYGON((178.95021197900937 -17.57911298342808,179.05767200098202 -17.57911298342808,179.05767200098202 -17.673020743331914,178.95021197900937 -17.673020743331914,178.95021197900937 -17.57911298342808))"

#parameter display_name="Mosaic Type" description="The type of mosaic." datatype="string" options=["max", "median"],
mosaic_type = "median"

#parameter display_name="Coordinate Reference System" description="The EPSG code for the CRS, for Fiji this will be 3460." datatype="string" options=["3460", "3832"],
crs = "3460"


### Create AOI

In [5]:
%%time
lat_extents, lon_extents = create_lat_lon(aoi_wkt)

CPU times: user 22.6 ms, sys: 0 ns, total: 22.6 ms
Wall time: 21.2 ms


In [6]:
# jupyteronly
## The code below renders a map that can be used to orient yourself with the region.
display_map(latitude = lat_extents, longitude = lon_extents)

In [7]:
#reprojection of AOI into input CRS and reformat
inProj  = Proj("+init=EPSG:4326")
outProj = Proj("+init=EPSG:"+crs)
min_lat, max_lat = (lat_extents) 
min_lon, max_lon = (lon_extents)
x_A, y_A = transform(inProj, outProj, min_lon, min_lat)
x_B, y_B = transform(inProj, outProj, max_lon, max_lat)
lat_range = (y_A, y_B)
lon_range = (x_A, x_B)
print(lat_range)
print(lon_range)

(3872595.635333245, 3880583.3719419655)
(1974816.6091847918, 1981238.1415149975)


## Translate inputs and load data

In [8]:
allmeasurements = ["green","red","blue","nir","swir1","swir2"]
water_measurements = ["watermask", "waterprob"]

def create_product_measurement(platform):
    if platform  in ["SENTINEL_2"]:
        product = 's2_esa_sr_granule'
        measurements = allmeasurements + ["coastal_aerosol","scene_classification"]
        water_product = 's2_water_mlclassification'
    elif platform in ["LANDSAT_8"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls8_usgs_sr_scene'
        water_product = 'ls8_water_mlclassification'
    elif platform in ["LANDSAT_7"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls7_usgs_sr_scene'
        water_product = 'ls7_water_mlclassification'
    elif platform in ["LANDSAT_5"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls5_usgs_sr_scene'
        water_product = 'ls5_water_classification'
    elif platform in ["LANDSAT_4"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls4_usgs_sr_scene'
        water_product = 'ls4_water_classification'
    else:
        print("invalid platform")
    return product, measurements, water_product

In [9]:
%%time
baseline_product, baseline_measurement, baseline_water_product = create_product_measurement(baseline_platform)
analysis_product, analysis_measurement, analysis_water_product = create_product_measurement(analysis_platform)

CPU times: user 22 µs, sys: 4 µs, total: 26 µs
Wall time: 34.6 µs


In [10]:
#create resolution
resolution = (-res, res)

In [11]:
dask_chunks=dict(
    #time = 1,
    x = 1000,
    y = 1000
)

In [12]:
%%time
#format dates
def createDate(inputStart, inputEnd):
    start = datetime.strptime(inputStart, '%Y-%m-%d')
    end = datetime.strptime(inputEnd, '%Y-%m-%d')
    startDates = start.date()
    endDates = end.date()
    time_period = (startDates, endDates)
    return time_period

baseline_time_period = createDate(baseline_time_start, baseline_time_end)
analysis_time_period = createDate(analysis_time_start, analysis_time_end)

print(baseline_time_period)

(datetime.date(2010, 1, 1), datetime.date(2010, 12, 31))
CPU times: user 0 ns, sys: 4.23 ms, total: 4.23 ms
Wall time: 3.69 ms


In [13]:
# Create the 'query' dictionary object, which contains the longitudes, latitudes 
query = {
    'y': lat_range,
    'x': lon_range,
    'output_crs': "EPSG:"+crs,  
    'resolution': resolution,
    'dask_chunks': dask_chunks,
    'crs': "EPSG:"+crs,  
}

### Load data based on cube query

In [14]:
baseline_ds = dc.load(
    time = baseline_time_period,
    platform = baseline_platform,
    product = baseline_product,
    measurements = baseline_measurement,
    **query
)

baseline_ds

,Array,Chunk
Bytes,3.21 MB,114.81 kB
Shape,"(28, 267, 215)","(1, 267, 215)"
Count,56 Tasks,28 Chunks
Type,int16,numpy.ndarray
,Array,Chunk
Bytes,3.21 MB,114.81 kB
Shape,"(28, 267, 215)","(1, 267, 215)"
Count,56 Tasks,28 Chunks
Type,int16,numpy.ndarray
,Array,Chunk


In [15]:
analysis_ds = dc.load(
    time = analysis_time_period,
    platform = analysis_platform,
    product = analysis_product,
    measurements = analysis_measurement,
    **query
)

analysis_ds

,Array,Chunk
Bytes,5.28 MB,114.81 kB
Shape,"(46, 267, 215)","(1, 267, 215)"
Count,92 Tasks,46 Chunks
Type,int16,numpy.ndarray
,Array,Chunk
Bytes,5.28 MB,114.81 kB
Shape,"(46, 267, 215)","(1, 267, 215)"
Count,92 Tasks,46 Chunks
Type,int16,numpy.ndarray
,Array,Chunk


### Check if loads are empty

In [16]:
def is_dataset_empty(ds:xr.Dataset) -> bool:
    checks_for_empty = [
                        lambda x: len(x.dims) == 0,      #Dataset has no dimensions
                        lambda x: len(x.data_vars) == 0  #Dataset no variables 
                       ]
    for f in checks_for_empty:
         if f(ds) == True:
                return True
    return False

In [17]:
if is_dataset_empty(baseline_ds): raise Exception("DataCube Load returned an empty Dataset." +  
                                               "Please check load parameters for Baseline Dataset!")

In [18]:
if is_dataset_empty(analysis_ds): raise Exception("DataCube Load returned an empty Dataset." +  
                                               "Please check load parameters for Analysis Dataset!")

#### Create Cloud Mask
Generating boolean masks that highlight valid pixels
Pixels must be cloud-free over land or water to be considered

In [19]:
def look_up_clean(platform, ds):
    if platform  in ["SENTINEL_2"]:
        good_quality = (
            (ds.scene_classification == 4) | # clear
            (ds.scene_classification == 5) | 
            (ds.scene_classification == 7) | 
            (ds.scene_classification == 2) | 
            (ds.scene_classification == 6)  #water
        )
    elif platform in ["LANDSAT_8"]:  
        good_quality = (
            (ds.pixel_qa == 322)  | # clear
            (ds.pixel_qa == 386)  |
            (ds.pixel_qa == 834)  |
            (ds.pixel_qa == 898)  |
            (ds.pixel_qa == 1346) |
            (ds.pixel_qa == 324)  | # water
            (ds.pixel_qa == 388)  |
            (ds.pixel_qa == 836)  |
            (ds.pixel_qa == 900)  |
            (ds.pixel_qa == 1348)
        )
    elif platform in ["LANDSAT_7", "LANDSAT_5", "LANDSAT_4"]:    
        good_quality = (
            (ds.pixel_qa == 66)  | # clear
            (ds.pixel_qa == 130) |
            (ds.pixel_qa == 68)  | # water
            (ds.pixel_qa == 132)  
        )
    else:
        print("invalid platform")
    return good_quality

In [20]:
%%time
b_good_quality = look_up_clean(baseline_platform, baseline_ds)
a_good_quality = look_up_clean(analysis_platform, analysis_ds)

CPU times: user 66.2 ms, sys: 0 ns, total: 66.2 ms
Wall time: 65.7 ms


In [21]:
baseline_ds_m1 = baseline_ds.where(b_good_quality)
analysis_ds_m1 = analysis_ds.where(a_good_quality)

baseline_ds_m1

,Array,Chunk
Bytes,12.86 MB,459.24 kB
Shape,"(28, 267, 215)","(1, 267, 215)"
Count,336 Tasks,28 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,12.86 MB,459.24 kB
Shape,"(28, 267, 215)","(1, 267, 215)"
Count,336 Tasks,28 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [22]:
#carry out additional mask
maskoutofrange_b = ((baseline_ds_m1> 0) & (baseline_ds_m1 <= 10000))
baseline_ds_m2 = baseline_ds_m1.where(maskoutofrange_b != 0)
baseline_ds = baseline_ds_m2

In [23]:
#carry out additional mask
maskoutofrange_a = ((analysis_ds_m1> 0) & (analysis_ds_m1 <= 10000))
analysis_ds_m2 = analysis_ds_m1.where(maskoutofrange_a != 0)
analysis_ds = analysis_ds_m2

### Perform Mosaic
 Use clean masks in a time series composite

In [24]:
#add in geomedian - get rid of others
mosaic_function = {"median": create_median_mosaic,
                   "max_ndvi": create_max_ndvi_mosaic}

In [25]:
new_compositor = mosaic_function[mosaic_type]
new_compositor

<function datacube_utilities.dc_mosaic.create_median_mosaic(dataset_in, clean_mask=None, no_data=nan, dtype=None, **kwargs)>

In [26]:
if mosaic_type == "median":
    baseline_composite = new_compositor(baseline_ds, clean_mask = b_good_quality)
    analysis_composite = new_compositor(analysis_ds, clean_mask = a_good_quality)
else:    
    baseline_composite = dask.delayed(new_compositor)(baseline_ds, clean_mask = b_good_quality)
    analysis_composite = dask.delayed(new_compositor)(analysis_ds, clean_mask = a_good_quality)

baseline_composite

<xarray.Dataset>
Dimensions:      (x: 215, y: 267)
Coordinates:
  * y            (y) float64 3.881e+06 3.881e+06 ... 3.873e+06 3.873e+06
  * x            (x) float64 1.975e+06 1.975e+06 ... 1.981e+06 1.981e+06
    spatial_ref  int32 3460
Data variables:
    green        (y, x) float64 dask.array<chunksize=(267, 215), meta=np.ndarray>
    red          (y, x) float64 dask.array<chunksize=(267, 215), meta=np.ndarray>
    blue         (y, x) float64 dask.array<chunksize=(267, 215), meta=np.ndarray>
    nir          (y, x) float64 dask.array<chunksize=(267, 215), meta=np.ndarray>
    swir1        (y, x) float64 dask.array<chunksize=(267, 215), meta=np.ndarray>
    swir2        (y, x) float64 dask.array<chunksize=(267, 215), meta=np.ndarray>
    pixel_qa     (y, x) float64 dask.array<chunksize=(267, 215), meta=np.ndarray>

### Mask Water

In [27]:
def loadWaterMask(productInput):
    if productInput in ["s2_water_mlclassification", "ls8_water_mlclassification", "ls7_water_mlclassification"]:
        water_scenes = dc.load(product=productInput,
              measurements = ["watermask", "waterprob"],
               **query)
        if is_dataset_empty(water_scenes):
            print(productInput, 'is empty')
        #change clouds to no data value
        else:
            water_classes = water_scenes.where(water_scenes >= 0)
            good_quality_water = (
               (water_scenes.watermask >= 0) & # no data
                (
                 (water_scenes.waterprob <= 5) |
                    (water_scenes.waterprob >= 100-5)
                )
                )
            water_classes = water_scenes.where(good_quality_water)
            water_classes['waterprob'] = (100-water_classes['waterprob']) # assign nodata vals consistent w/ other prods
            product_data = water_classes
    elif productInput in ["ls4_water_classification", "ls5_water_classification"]:
        water_scenes = dc.load(product=productInput,
                               measurements = ["water"],
                                   **query)
        if is_dataset_empty(water_scenes):
            print(productInput, 'is empty')
        else:
            water_classes1 = water_scenes.where(water_scenes != -9999)
            water_classes1['watermask'] = water_classes1['water']
            water_classes = water_classes1.drop(['water'])
            product_data = water_classes
    else:
        print('invalid platform')
    return product_data

In [28]:
water_scenes_baseline = loadWaterMask(baseline_water_product)
water_scenes_analysis = loadWaterMask(analysis_water_product)

In [29]:
%%time
water_composite_base = water_scenes_baseline.watermask.mean(dim='time')
water_composite_analysis = water_scenes_analysis.watermask.mean(dim='time')

CPU times: user 16.7 ms, sys: 4.1 ms, total: 20.8 ms
Wall time: 20.5 ms


In [30]:
%%time
baseline_composite = baseline_composite.where((baseline_composite != np.nan) & (water_composite_base <= waterThresh))
analysis_composite = analysis_composite.where((analysis_composite != np.nan) & (water_composite_analysis <= waterThresh))

CPU times: user 64.3 ms, sys: 37 µs, total: 64.4 ms
Wall time: 63.8 ms


In [31]:
#plotting settings
def aspect_ratio_helper(ds, fixed_width = 15):
        y,x = ds.values.shape
        width = fixed_width
        height = y * (fixed_width / x)
        return (width, height)
RdYlGn.set_bad('black',1.)
Greens.set_bad('black',1.)

# NDVI Anomaly

In [32]:
%%time
#calculate NDVI
ndvi_baseline_composite = NDVI(baseline_composite)
ndvi_analysis_composite = NDVI(analysis_composite)

CPU times: user 17.6 ms, sys: 211 µs, total: 17.8 ms
Wall time: 29.7 ms


In [33]:
%%time
#calculate ndvi anomaly
ndvi_anomaly = ndvi_analysis_composite - ndvi_baseline_composite

CPU times: user 3.29 ms, sys: 0 ns, total: 3.29 ms
Wall time: 3.26 ms


In [34]:
%%time
ndvi_anomaly_output = ndvi_anomaly.compute()

RasterioIOError: '/vsis3/public-eo-data/common_sensing/fiji/mlwatermasks/LC08_L1TP_074073_20130925_mlwater/LC08_L1TP_074073_20130925_watermask.tif' not recognized as a supported file format.

>#### NDVI Anomaly

This product shows the following ...<br>
BLACK = Cloud or Pixels NOT in the baseline threshold range<br>
GREEN = Pixels with an increase in NDVI<br>
RED = Pixels with a decrease in NDVI<br>

In [60]:
# jupyteronly
plt.figure(figsize = (10,8))
ndvi_anomaly_output.plot(vmin=-1, vmax=1, cmap = RdYlGn)

NameError: name 'ndvi_anomaly_output' is not defined

<Figure size 720x576 with 0 Axes>

### Export
Export output as a Cloud Optimised Geotiff, also option available for regular Geotiff.

In [ ]:
#Write as Cog
write_cog(geo_im=ndvi_anomaly_output,
          fname='ndvi_anomaly_output.tif',
          overwrite=True)

In [ ]:
#export geotiff
#ndvi_anomaly_export = xr.DataArray.to_dataset(ndvi_anomaly_output, dim = None, name = 'ndvi_anomaly')
#write_geotiff_from_xr('ndvi_anomaly_10m.tiff', ndvi_anomaly_export, ["ndvi_anomaly"], crs=output_projection, x_coord = 'x', y_coord = 'y')

In [ ]:
['ndvi_anomaly_output']

---